# ECON546 Lab03 2018 Spring

## Tutorial

https://www.youtube.com/watch?v=YMt5K68ZvjQ

http://davidbraudt.com/stata-videos/

https://www.youtube.com/watch?v=QaI_a_l2jqo

http://davidbraudt.com/wp-content/uploads/2016/11/Command-List-for-Fall-2015-Workshop.pdf

## Macro

### 1 Macros
A macro is simply a name associated with some text. Macros can be local or global in scope.

Local macros have names of up to 31 characters and are known only in the **current context** (the console, a do file, or a program).

**Warning**: Evaluating a macro that doesn't exist is not an error; it just returns an empty string. So be careful to spell macro names correctly. 


```stata
local age "age20to24 age25to29 age30to34 age35to39 age40to44 age45to49"
local controls \`age' income education
```


ref:

http://data.princeton.edu/stata/programming.html


https://www.youtube.com/watch?v=Ovk0CiTxMRI

https://www.youtube.com/watch?v=WsqvGdqzknY

http://jearl.faculty.arizona.edu/sites/jearl.faculty.arizona.edu/files/Intro%20to%20loops%2C%20Year%202.pdf

https://www.ucl.ac.uk/pcph/research-groups-themes/thin-pub/research_presentations/Collier_looping_STATA

## For loop

### Looping Over Sequences of Numbers
The basic looping command takes the form

```stata
forvalues number = sequence {
    ... body of loop using `number' ...
}
```

Here forvalues is a keyword, number is the name of a local macro that will be set to each number in the sequence, and sequence is a range of values which can have the form

- min/max to indicate a sequence of numbers from min to max in steps of one, for example 1/3 yields 1, 2 and 3, or
- first(step)last which yields a sequence from first to last in steps of size step. For example 15(5)50 yields 15,20,25,30,35,40,45 and 50.

The opening left brace must be the last thing on the first line (other than comments), and the loop must be closed by a matching right brace on a line all by itself. The loop is executed once for each value in the sequence with your local macro number (or whatever you called it) holding the value.

### Looping Over Elements in a List

```stata
foreach item in a-list-of-things {
    ... body of loop using `item' ...
}
```



ref:

http://data.princeton.edu/stata/programming.html

https://www.youtube.com/watch?v=9S54_YLQ7WI

In [ ]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import statsmodels.formula.api as smf
import ipystata

In [ ]:
%%stata -o life_df
sysuse lifeexp.dta
summarize

In [ ]:
%%stata -o labor_df
clear
use "D:\onedrive\One drive\546-2018\lab\lab03\546lab3.dta"
capture log close
log using "D:\onedrive\One drive\546-2018\lab\lab03\546mylab3.log", replace
set seed 1234567
g age2=age^2 // creating the square of age

In [ ]:
%%stata -d labor_df --graph 
graph twoway (scatter logwr educ) (lfit logwr educ)
graph save "D:\onedrive\One drive\546-2018\lab\lab03\graph1.gph", replace

In [ ]:
%%stata -d labor_df

label variable inlf "Women’s participation =1 yes, =0 no"

label variable nclt18 "Number of children between 4 and 18"

label variable age "Age of woman"

notes logwr: this is the natural logarithm of the wage rate

In [ ]:
%%stata -d labor_df
sum inlf if age<=30 | age>=50

In [ ]:
%%stata -d labor_df
g n3=0
replace n3=1 if nclt3 != 0
sum n3

In [ ]:
%%stata -d labor_df
g n4 = nclt3 != 0
sum n4

## Generate random variables

#### Exponetial distribution

The general approach is to take the cumulative distribution function
F(x) and solve for x in terms of F.  Then replace F with uniform() and
you are done:
```
F=1-exp(-lambda*x)
1-F=exp(-lambda*x)
-lambda*x=ln(1-F)
x=-(1/lambda)*ln(1-F)
```
so for lambda=1, you can
```
g e1=-ln(1-uniform())
```

ref:

https://www.stata.com/statalist/archive/2008-02/msg00857.html

In [ ]:
%%stata -d labor_df
g x1=rnormal()
g x2=rnormal(50,5)
sum x1 x2

In [ ]:
%%stata -d labor_df
local variables nclt18 age age2 educ logwr
regress inlf nclt3 `variables'

In [ ]:
model = smf.ols(formula = 'inlf ~ nclt3 + nclt18+ age+ age2 +educ +logwr',
                data = labor_df)
results = model.fit()
print(results.summary())

In [ ]:
%%stata -d labor_df

local variables nclt18 age age2 educ logwr

// illustrating a forvalues loop

forvalues loop=0/2 {
regress inlf `variables' if nclt3==`loop'
}



### Creating Dummy Variables
creating dummy variables to represent age groups. Stata 11 introduced factor variables and Stata 13 improved the labeling of tables of estimates, drastically reducing the need to "roll your own" dummies, but the code remains instructive.

This will create dummy variables age20to24 to age45to49. The way the loop works is that the local macro bot will take values between 20 and 45 in steps of 5 (hence 20, 25, 30, 35, 40, and 45), the lower bounds of the age groups.

Inside the loop we create a local macro top to represent the upper bounds of the age groups, which equals the lower bound plus 4. The first time through the loop bot is 20, so top is 24. We use an equal sign to store the result of adding 4 to bot.

In [ ]:
%%stata -d labor_df
forvalues bot = 20(5)45 {
    local top = `bot' + 4
    gen age`bot'to`top' = age >= `bot' & age <= `top'
}

sum age20to24 age25to29 age30to34 age35to39 age40to44 age45to49


In [ ]:
%%stata -d labor_df
local controls nclt18 age age2 educ logwr

// illustrating a foreach loop

foreach control of local controls {
    display "`control'"
}

In [ ]:
%%stata 

// illustrating a foreach loop

foreach year of numlist 1980 1985 1995 {
    display "`year'"
}

In [ ]:
%%stata -d labor_df
local variables nclt18 age age2 educ logwr

// illustrating a while loop

local k=1
while `k'<=2 {
regress inlf `variables' if nclt3==`k'
local k=`k'+1
}